# Summary of the benchmark

The results can be compared with [this article](https://arxiv.org/pdf/2403.08035) in which GPT-3.5 achieves $\text{acc}=0.89$ and $F_1=0.93$ in *English Hate Speech Detection*. We use newer models in this benchmark.

## Metrics

Our task is a binary classification. We aim at reaching a ground truth $f:E\to\{0,1\}$ which we will estimate with $\hat f:E\to\{0,1\}$.

### Confusion Matrix Notations (Binary Classification)

|              | Predicted 0 | Predicted 1 |
| ------------ | ----------- | ----------- |
| **Actual 0** | TN          | FP          |
| **Actual 1** | FN          | TP          |

* **TP** = True Positives (predicted 1, actual 1)
* **TN** = True Negatives (predicted 0, actual 0)
* **FP** = False Positives (predicted 1, actual 0)
* **FN** = False Negatives (predicted 0, actual 1)

### Metrics 

- **Precision\_0** : The proportion of predicted class 0 that is actually class 0 : $$\text{Precision}\_0 = \frac{\text{TN}}{\text{TN} + \text{FN}}$$
- **Recall\_0** : The proportion of actual class 0 correctly predicted as class 0 : $$\text{Recall}\_0 = \frac{\text{TN}}{\text{TN} + \text{FP}}$$
- **F1\_0** : The harmonic mean of Precision\_0 and Recall\_0 : $$\text{F1}_0 = 2 \cdot \frac{\text{Precision}\_0 \times \text{Recall}\_0}{\text{Precision}\_0 + \text{Recall}\_0}$$
- **Precision\_1** : The proportion of predicted class 1 that is actually class 1 : $$\text{Precision}\_1 = \frac{\text{TN}}{\text{TN} + \text{FN}}$$
- **Recall\_1** : The proportion of actual class 1 correctly predicted as class 1 : $$\text{Recall}\_1 = \frac{\text{TN}}{\text{TN} + \text{FP}}$$
- **F1\_1** : The harmonic mean of Precision\_1 and Recall\_1 : $$\text{F1}_0 = 2 \cdot \frac{\text{Precision}\_1 \times \text{Recall}\_1}{\text{Precision}\_1 + \text{Recall}\_1}$$
- **Accuracy** : The proportion of all correct predictions : $$\text{Accuracy} = \frac{\text{TP} + \text{TN}}{\text{TP} + \text{TN} + \text{FP} + \text{FN}}$$
- **ROC AUC** : Area under the **Receiver Operating Characteristic** curve. It evaluates the tradeoff between True Positive Rate (TPR) and False Positive Rate (FPR) over all thresholds : $$\text{TPR (Recall)} = \frac{\text{TP}}{\text{TP} + \text{FN}}, \quad 
\text{FPR} = \frac{\text{FP}}{\text{FP} + \text{TN}}$$

## Libraries

In [18]:
import pandas as pd 
from pathlib import Path
import os
from rich.console import Console
from rich.table import Table
from rich.panel import Panel
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score
)

## Global variables

In [ ]:
ROOT = Path('../..')
DATA_PATH = ROOT / "data"
BENCHMARK_PATH = DATA_PATH / "benchmark"

console = Console()

## Load predictions

In [20]:
to_exclude = [
    "benchmark.csv",
    "benchmark_balanced_subset.csv",
    "checkpoint"
]

In [21]:
df_benchmark = pd.read_csv(BENCHMARK_PATH / "benchmark_balanced_subset.csv", encoding = 'utf-8')
len_benchmark = len(df_benchmark)
console.print(f"Loaded benchmark with {len_benchmark} entries.")

Loaded benchmark with 1388 entries.

In [22]:
dfs = []

for file in os.listdir(BENCHMARK_PATH):
    if file.endswith('.csv') and all(file_exclude not in file for file_exclude in to_exclude):
        file_path = BENCHMARK_PATH / file
        df = pd.read_csv(file_path, encoding='utf-8')
        df['file'] = file  
        assert len(df) == len_benchmark, f"Length mismatch for {file}: {len(df)} vs {len_benchmark}"
        assert "prediction" in df.columns, f"'prediction' column missing in {file}"
        assert "label" in df.columns, f"'prediction' column missing in {file}"
        dfs.append(df)

console.print(f"Loaded {len(dfs)} additional files from benchmark directory.")

Loaded 34 additional files from benchmark directory.

## Compute the metrics

In [23]:
results = []

for df in dfs:
    file_name = df['file'].iloc[0]
    y_true = df['label'].astype(int)
    y_pred = df['prediction']
    row = {"Model": file_name.replace('.csv', '').replace('_', ' ')}

    try:
        report = classification_report(y_true, y_pred, output_dict=True)
        row.update({
            "Precision_0": report['0']['precision'],
            "Recall_0": report['0']['recall'],
            "F1_0": report['0']['f1-score'],
            "Precision_1": report['1']['precision'],
            "Recall_1": report['1']['recall'],
            "F1_1": report['1']['f1-score'],
            "Accuracy": report['accuracy'],
        })
    except Exception as e:
        console.print(f"[red]Error computing classification report for {file_name}: {e}[/red]")

    try:
        roc_auc = roc_auc_score(y_true, y_pred)
        row["ROC_AUC"] = roc_auc
    except:
        row["ROC_AUC"] = None

    results.append(row)

In [24]:
# === Convert to DataFrame & Display as Rich Table ===
summary_df = pd.DataFrame(results)
summary_df = summary_df.sort_values(by="Accuracy", ascending=False) 

# === Print Table in Rich ===
rich_table = Table(title="Benchmark Summary for All Models", show_lines=True)
for col in summary_df.columns:
    rich_table.add_column(col, justify="center")

for _, row in summary_df.iterrows():
    rich_table.add_row(*[f"{x:.3f}" if isinstance(x, float) else str(x) for x in row])

console.print(rich_table)

                                         Benchmark Summary for All Models                                          
┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┓
┃          Model           ┃ Precision_0 ┃ Recall_0 ┃ F1_0  ┃ Precision_1 ┃ Recall_1 ┃ F1_1  ┃ Accuracy ┃ ROC_AUC ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━┩
│       gpt-4o-mini        │    0.962    │  0.771   │ 0.856 │    0.809    │  0.970   │ 0.882 │  0.870   │  0.870  │
├──────────────────────────┼─────────────┼──────────┼───────┼─────────────┼──────────┼───────┼──────────┼─────────┤
│     gemini-2.0-flash     │    0.954    │  0.769   │ 0.852 │    0.807    │  0.963   │ 0.878 │  0.866   │  0.866  │
├──────────────────────────┼─────────────┼──────────┼───────┼─────────────┼──────────┼───────┼──────────┼─────────┤
│      mistral medium      │    0.924    │  0.787   │ 0.850 │    0.814    │  0.935   │ 0.871 │  0.861   │  0.861  │
├──────────────────────────┼─────────────┼──────────┼───────┼─────────────┼──────────┼───────┼──────────┼─────────┤
│    gpt-4o-mini simple    │    0.959    │  0.746   │ 0.840 │    0.792    │  0.968   │ 0.872 │  0.857   │  0.857  │
│          prompt          │             │          │       │             │          │       │          │         │
├──────────────────────────┼─────────────┼──────────┼───────┼─────────────┼──────────┼───────┼──────────┼─────────┤
│      mistral large       │    0.953    │  0.752   │ 0.841 │    0.795    │  0.963   │ 0.871 │  0.857   │  0.857  │
├──────────────────────────┼─────────────┼──────────┼───────┼─────────────┼──────────┼───────┼──────────┼─────────┤
│      gemini-1.5-pro      │    0.942    │  0.755   │ 0.838 │    0.796    │  0.954   │ 0.868 │  0.854   │  0.854  │
├──────────────────────────┼─────────────┼──────────┼───────┼─────────────┼──────────┼───────┼──────────┼─────────┤
│          gpt-4o          │    0.990    │  0.696   │ 0.817 │    0.766    │  0.993   │ 0.864 │  0.844   │  0.844  │
├──────────────────────────┼─────────────┼──────────┼───────┼─────────────┼──────────┼───────┼──────────┼─────────┤
│    deepseek-reasoner     │    0.967    │  0.709   │ 0.818 │    0.770    │  0.976   │ 0.861 │  0.842   │  0.842  │
├──────────────────────────┼─────────────┼──────────┼───────┼─────────────┼──────────┼───────┼──────────┼─────────┤
│      gpt-3.5-turbo       │    0.899    │  0.767   │ 0.827 │    0.796    │  0.914   │ 0.851 │  0.840   │  0.840  │
├──────────────────────────┼─────────────┼──────────┼───────┼─────────────┼──────────┼───────┼──────────┼─────────┤
│      deepseek-chat       │    0.983    │  0.683   │ 0.806 │    0.757    │  0.988   │ 0.858 │  0.836   │  0.836  │
├──────────────────────────┼─────────────┼──────────┼───────┼─────────────┼──────────┼───────┼──────────┼─────────┤
│      mistral small       │    0.954    │  0.695   │ 0.804 │    0.760    │  0.967   │ 0.851 │  0.831   │  0.831  │
├──────────────────────────┼─────────────┼──────────┼───────┼─────────────┼──────────┼───────┼──────────┼─────────┤
│  mistral medium simple   │    0.971    │  0.679   │ 0.799 │    0.753    │  0.980   │ 0.852 │  0.829   │  0.829  │
│          prompt          │             │          │       │             │          │       │          │         │
├──────────────────────────┼─────────────┼──────────┼───────┼─────────────┼──────────┼───────┼──────────┼─────────┤
│ gemini-2.0-flash simple  │    0.971    │  0.671   │ 0.794 │    0.749    │  0.980   │ 0.849 │  0.826   │  0.826  │
│          prompt          │             │          │       │             │          │       │          │         │
├──────────────────────────┼─────────────┼──────────┼───────┼─────────────┼──────────┼───────┼──────────┼─────────┤
│         o4-mini          │    0.862    │  0.767   │ 0.812 │    0.790    │  0.878   │ 0.831 │  0.822   │  0.822  │
├──────────────────────────┼─────────────┼──────────┼───